In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta

In [2]:
train = pd.read_csv('datasets/train.csv')
test = pd.read_csv('datasets/test.csv')
train['Split'] = 'Train'
test['Split'] = 'Test'
test.head()

,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Split
0,28,40,2012-01-06,False,50.43,3.599,5155.86,19549.51,218.29,636.73,5218.50,130.157516,12.187,A,206302,Test
1,18,93,2012-03-09,False,38.10,3.876,7410.18,368.19,25.46,1852.39,2332.07,137.506690,8.075,B,120653,Test
2,38,4,2011-05-27,False,76.67,4.087,NaN,NaN,NaN,NaN,NaN,129.062355,13.736,C,39690,Test
3,29,33,2010-03-12,False,43.43,2.805,NaN,NaN,NaN,NaN,NaN,131.784000,10.064,B,93638,Test
4,13,52,2010-10-29,False,45.12,2.841,NaN,NaN,NaN,NaN,NaN,126.436419,7.795,A,219622,Test


In [3]:
t_len = len(train)
df = pd.concat([train,test],axis=0)
df.tail()

C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,CPI,Date,Dept,Fuel_Price,IsHoliday,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,Size,Split,Store,Temperature,Type,Unemployment,Weekly_Sales
139114,129.826833,2011-11-18,91,3.669,False,86.59,6.00,NaN,NaN,3220.86,39690,Test,33,63.35,A,8.010,NaN
139115,130.645793,2012-03-02,85,3.242,False,25204.67,2436.75,167.88,16066.76,7289.69,219622,Test,13,32.36,A,6.104,NaN
139116,211.855267,2011-01-28,91,3.010,False,NaN,NaN,NaN,NaN,NaN,42988,Test,30,44.04,C,8.028,NaN
139117,197.655186,2012-06-08,32,3.741,False,8483.84,361.80,77.99,1640.00,6841.59,203007,Test,32,71.14,A,8.090,NaN
139118,141.843393,2012-04-13,33,4.187,False,5139.48,4770.63,53.84,1532.47,4176.90,204184,Test,27,52.22,A,8.253,NaN


In [4]:
df.loc[df['Weekly_Sales'] >240000,"Date"].value_counts()

2011-11-25    15
2010-11-26    14
2010-12-24     8
2011-12-23     4
2010-12-17     2
2011-08-26     1
2010-02-05     1
Name: Date, dtype: int64

In [5]:
df.isnull().sum()

CPI                  0
Date                 0
Dept                 0
Fuel_Price           0
IsHoliday            0
MarkDown1       270889
MarkDown2       310322
MarkDown3       284479
MarkDown4       286603
MarkDown5       270138
Size                 0
Split                0
Store                0
Temperature          0
Type                 0
Unemployment         0
Weekly_Sales    139119
dtype: int64

In [6]:
df = df.assign(md1_present = df.MarkDown1.notnull())
df = df.assign(md2_present = df.MarkDown2.notnull())
df = df.assign(md3_present = df.MarkDown3.notnull())
df = df.assign(md4_present = df.MarkDown4.notnull())
df = df.assign(md5_present = df.MarkDown5.notnull())

In [7]:
df.fillna(0, inplace=True)

In [8]:
df['Type'] = 'Type_' + df['Type'].map(str)
df['Store'] = 'Store_' + df['Store'].map(str)
df['Dept'] = 'Dept_' + df['Dept'].map(str)
df['IsHoliday'] = 'IsHoliday_' + df['IsHoliday'].map(str)

In [9]:
type_dummies = pd.get_dummies(df['Type'])
store_dummies = pd.get_dummies(df['Store'])
dept_dummies = pd.get_dummies(df['Dept'])
holiday_dummies = pd.get_dummies(df['IsHoliday'])

In [10]:
df['DateType'] = [datetime.strptime(date, '%Y-%m-%d').date() for date in df['Date'].astype(str).values.tolist()]
df['Month'] = [date.month for date in df['DateType']]
df['Month'] = 'Month_' + df['Month'].map(str)
Month_dummies = pd.get_dummies(df['Month'] )

In [11]:
df['Black_Friday'] = np.where((df['DateType']==datetime(2010, 11, 26).date()) | (df['DateType']==datetime(2011, 11, 25).date()), 'yes', 'no')
df['Pre_christmas'] = np.where((df['DateType']==datetime(2010, 12, 23).date()) | (df['DateType']==datetime(2010, 12, 24).date()) | (df['DateType']==datetime(2011, 12, 23).date()) | (df['DateType']==datetime(2011, 12, 24).date()), 'yes', 'no')
df['Black_Friday'] = 'Black_Friday_' + df['Black_Friday'].map(str)
df['Pre_christmas'] = 'Pre_christmas_' + df['Pre_christmas'].map(str)
Black_Friday_dummies = pd.get_dummies(df['Black_Friday'] )
Pre_christmas_dummies = pd.get_dummies(df['Pre_christmas'] )

In [12]:
df = pd.concat([df,holiday_dummies,Pre_christmas_dummies,Black_Friday_dummies],axis=1)

In [13]:
medians = pd.DataFrame({'Median Sales' :df.loc[df['Split']=='Train'].groupby(by=['Type','Dept','Store','Month','IsHoliday'])['Weekly_Sales'].median()}).reset_index()
medians.head()

,Type,Dept,Store,Month,IsHoliday,Median Sales
0,Type_A,Dept_1,Store_1,Month_1,IsHoliday_False,16894.400
1,Type_A,Dept_1,Store_1,Month_10,IsHoliday_False,29279.120
2,Type_A,Dept_1,Store_1,Month_11,IsHoliday_False,19300.025
3,Type_A,Dept_1,Store_1,Month_11,IsHoliday_True,19865.770
4,Type_A,Dept_1,Store_1,Month_12,IsHoliday_False,44912.860


In [14]:
df = df.merge(medians, how = 'outer', on = ['Type','Dept','Store','Month','IsHoliday'])

In [15]:
df['Median Sales'].fillna(df['Median Sales'].loc[df['Split']=='Train'].median(), inplace=True) 
df['Key'] = df['Type'].map(str)+df['Dept'].map(str)+df['Store'].map(str)+df['Date'].map(str)+df['IsHoliday'].map(str)

In [16]:
df['DateLagged'] = df['DateType']- timedelta(days=7)
df.head()

,CPI,Date,Dept,Fuel_Price,IsHoliday,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,...,Pre_christmas,IsHoliday_False,IsHoliday_True,Pre_christmas_no,Pre_christmas_yes,Black_Friday_no,Black_Friday_yes,Median Sales,Key,DateLagged
0,136.213613,2011-08-26,Dept_92,3.796,IsHoliday_False,0.00,0.0,0.00,0.00,0.00,...,Pre_christmas_no,1,0,1,0,1,0,93212.89,Type_ADept_92Store_262011-08-26IsHoliday_False,2011-08-19
1,138.281452,2012-08-24,Dept_92,3.874,IsHoliday_False,4507.08,33.6,149.46,1866.46,4685.63,...,Pre_christmas_no,1,0,1,0,1,0,93212.89,Type_ADept_92Store_262012-08-24IsHoliday_False,2012-08-17
2,136.054032,2011-08-05,Dept_92,3.903,IsHoliday_False,0.00,0.0,0.00,0.00,0.00,...,Pre_christmas_no,1,0,1,0,1,0,93212.89,Type_ADept_92Store_262011-08-05IsHoliday_False,2011-07-29
3,138.143774,2012-08-10,Dept_92,3.772,IsHoliday_False,5732.23,17.6,6.45,1524.63,2439.95,...,Pre_christmas_no,1,0,1,0,1,0,93212.89,Type_ADept_92Store_262012-08-10IsHoliday_False,2012-08-03
4,138.185710,2012-08-17,Dept_92,3.840,IsHoliday_False,2430.95,16.0,5.54,829.70,3038.77,...,Pre_christmas_no,1,0,1,0,1,0,93212.89,Type_ADept_92Store_262012-08-17IsHoliday_False,2012-08-10


In [17]:
sorted_df = df.sort_values(['Store', 'Dept','DateType'], ascending=[1, 1,1])
sorted_df = sorted_df.reset_index(drop=True)

In [18]:
sorted_df['LaggedSales'] = np.nan # Initialize column
sorted_df['LaggedAvailable'] = np.nan # Initialize column
last=df.loc[0] # intialize last row for first iteration. Doesn't really matter what it is
row_len = sorted_df.shape[0]
for index, row in sorted_df.iterrows():
    lag_date = row["DateLagged"]
    # Check if it matches by comparing last weeks value to the compared date 
    # And if weekly sales aren't 0
    if((last['DateType']== lag_date) & (last['Weekly_Sales']>0)): 
        sorted_df.set_value(index, 'LaggedSales',last['Weekly_Sales'])
        sorted_df.set_value(index, 'LaggedAvailable',1)
    else:
        sorted_df.set_value(index, 'LaggedSales',row['Median Sales']) # Fill with median
        sorted_df.set_value(index, 'LaggedAvailable',0)

    last = row #Remember last row for speed
    if(index%int(row_len/10)==0): #See progress by printing every 10% interval
        print(str(int(index*100/row_len))+'% loaded')

C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]
C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.
C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # This is added back by InteractiveShellApp.init_path()


0% loaded
10% loaded
20% loaded
30% loaded
40% loaded
50% loaded
60% loaded
70% loaded
80% loaded
90% loaded


In [19]:
sorted_df[['Dept', 'Store','DateType','LaggedSales','Weekly_Sales','Median Sales']].head()

,Dept,Store,DateType,LaggedSales,Weekly_Sales,Median Sales
0,Dept_1,Store_1,2010-02-05,20124.22,0.00,20124.220
1,Dept_1,Store_1,2010-02-12,41513.99,46039.49,41513.990
2,Dept_1,Store_1,2010-02-19,46039.49,0.00,20124.220
3,Dept_1,Store_1,2010-02-26,20124.22,19403.54,20124.220
4,Dept_1,Store_1,2010-03-05,19403.54,21827.90,20962.245


In [20]:
df = df.merge(sorted_df[['Dept', 'Store','DateType','LaggedSales','LaggedAvailable']], how = 'inner', on = ['Dept', 'Store','DateType'])

In [21]:
df['Sales_dif'] = df['Median Sales'] - df['LaggedSales']
df[['Dept', 'Store','DateType','LaggedSales','Weekly_Sales','Median Sales']].head()

,Dept,Store,DateType,LaggedSales,Weekly_Sales,Median Sales
0,Dept_92,Store_26,2011-08-26,93212.89,87235.57,93212.89
1,Dept_92,Store_26,2012-08-24,93212.89,88145.09,93212.89
2,Dept_92,Store_26,2011-08-05,80427.06,96906.53,93212.89
3,Dept_92,Store_26,2012-08-10,93890.07,111812.56,93212.89
4,Dept_92,Store_26,2012-08-17,111812.56,93212.89,93212.89


In [22]:
df['Difference'] = df['Median Sales'] - df['Weekly_Sales']
df.head()

,CPI,Date,Dept,Fuel_Price,IsHoliday,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,...,Pre_christmas_yes,Black_Friday_no,Black_Friday_yes,Median Sales,Key,DateLagged,LaggedSales,LaggedAvailable,Sales_dif,Difference
0,136.213613,2011-08-26,Dept_92,3.796,IsHoliday_False,0.00,0.0,0.00,0.00,0.00,...,0,1,0,93212.89,Type_ADept_92Store_262011-08-26IsHoliday_False,2011-08-19,93212.89,0.0,0.00,5977.32
1,138.281452,2012-08-24,Dept_92,3.874,IsHoliday_False,4507.08,33.6,149.46,1866.46,4685.63,...,0,1,0,93212.89,Type_ADept_92Store_262012-08-24IsHoliday_False,2012-08-17,93212.89,1.0,0.00,5067.80
2,136.054032,2011-08-05,Dept_92,3.903,IsHoliday_False,0.00,0.0,0.00,0.00,0.00,...,0,1,0,93212.89,Type_ADept_92Store_262011-08-05IsHoliday_False,2011-07-29,80427.06,1.0,12785.83,-3693.64
3,138.143774,2012-08-10,Dept_92,3.772,IsHoliday_False,5732.23,17.6,6.45,1524.63,2439.95,...,0,1,0,93212.89,Type_ADept_92Store_262012-08-10IsHoliday_False,2012-08-03,93890.07,1.0,-677.18,-18599.67
4,138.185710,2012-08-17,Dept_92,3.840,IsHoliday_False,2430.95,16.0,5.54,829.70,3038.77,...,0,1,0,93212.89,Type_ADept_92Store_262012-08-17IsHoliday_False,2012-08-10,111812.56,1.0,-18599.67,0.00


In [23]:
selector = [
    #'Month',
    'CPI',
    'Fuel_Price',
    'MarkDown1',
    'MarkDown2',
    'MarkDown3',
    'MarkDown4',
    'MarkDown5',
    'Size',
    'Temperature',
    'Unemployment',
    
    
    
    'md1_present',
    'md2_present',
    'md3_present',
    'md4_present',
    'md5_present',

    'IsHoliday_False',
    'IsHoliday_True',
    'Pre_christmas_no',
    'Pre_christmas_yes',
    'Black_Friday_no',
    'Black_Friday_yes',    
    'LaggedSales',
    'Sales_dif',
    'LaggedAvailable'
    ]
display(df[selector].describe())
display(df[selector].head())

,CPI,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,Size,Temperature,Unemployment,IsHoliday_False,IsHoliday_True,Pre_christmas_no,Pre_christmas_yes,Black_Friday_no,Black_Friday_yes,LaggedSales,Sales_dif,LaggedAvailable
count,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000
mean,171.201947,3.361027,2590.074819,879.974298,468.087665,1083.132268,1662.772385,136727.915739,60.090059,7.960289,0.929642,0.070358,0.985808,0.014192,0.985865,0.014135,15907.380029,-179.734399,0.655172
std,39.159276,0.458515,6052.385934,5084.538801,5528.873453,3894.529945,4207.629321,60980.583328,18.447931,1.863296,0.255750,0.255750,0.118283,0.118283,0.118049,0.118049,22521.689433,5538.988872,0.475313
min,126.064000,2.472000,0.000000,-265.760000,-29.100000,0.000000,0.000000,34875.000000,-2.060000,3.879000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1750.000000,-526108.280000,0.000000
25%,132.022667,2.933000,0.000000,0.000000,0.000000,0.000000,0.000000,93638.000000,46.680000,6.891000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,2114.310000,-118.608750,0.000000
50%,182.318780,3.452000,0.000000,0.000000,0.000000,0.000000,0.000000,140167.000000,62.090000,7.866000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,7583.440000,0.000000,1.000000
75%,212.416993,3.738000,2809.050000,2.200000,4.540000,425.290000,2168.040000,202505.000000,74.280000,8.572000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,20107.905000,114.821250,1.000000
max,227.232807,4.468000,88646.760000,104519.540000,141630.610000,67474.850000,108519.280000,219622.000000,100.140000,14.313000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,693099.360000,557005.550000,1.000000


,CPI,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,Size,Temperature,Unemployment,...,md5_present,IsHoliday_False,IsHoliday_True,Pre_christmas_no,Pre_christmas_yes,Black_Friday_no,Black_Friday_yes,LaggedSales,Sales_dif,LaggedAvailable
0,136.213613,3.796,0.00,0.0,0.00,0.00,0.00,152513,61.10,7.767,...,False,1,0,1,0,1,0,93212.89,0.00,0.0
1,138.281452,3.874,4507.08,33.6,149.46,1866.46,4685.63,152513,62.08,7.405,...,True,1,0,1,0,1,0,93212.89,0.00,1.0
2,136.054032,3.903,0.00,0.0,0.00,0.00,0.00,152513,63.16,7.767,...,False,1,0,1,0,1,0,80427.06,12785.83,1.0
3,138.143774,3.772,5732.23,17.6,6.45,1524.63,2439.95,152513,67.01,7.405,...,True,1,0,1,0,1,0,93890.07,-677.18,1.0
4,138.185710,3.840,2430.95,16.0,5.54,829.70,3038.77,152513,65.54,7.405,...,True,1,0,1,0,1,0,111812.56,-18599.67,1.0


In [24]:
train = df.loc[df['Split']=='Train']
test = df.loc[df['Split']=='Test']
test.head()

,CPI,Date,Dept,Fuel_Price,IsHoliday,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,...,Pre_christmas_yes,Black_Friday_no,Black_Friday_yes,Median Sales,Key,DateLagged,LaggedSales,LaggedAvailable,Sales_dif,Difference
7,132.661613,2010-08-20,Dept_92,2.796,IsHoliday_False,0.0,0.0,0.0,0.0,0.0,...,0,1,0,93212.89,Type_ADept_92Store_262010-08-20IsHoliday_False,2010-08-13,80498.48,1.0,12714.41,93212.89
8,136.144129,2011-08-12,Dept_92,3.880,IsHoliday_False,0.0,0.0,0.0,0.0,0.0,...,0,1,0,93212.89,Type_ADept_92Store_262011-08-12IsHoliday_False,2011-08-05,96906.53,1.0,-3693.64,93212.89
9,132.614193,2010-08-06,Dept_92,2.792,IsHoliday_False,0.0,0.0,0.0,0.0,0.0,...,0,1,0,93212.89,Type_ADept_92Store_262010-08-06IsHoliday_False,2010-07-30,93212.89,0.0,0.00,93212.89
10,132.693226,2010-08-27,Dept_92,2.770,IsHoliday_False,0.0,0.0,0.0,0.0,0.0,...,0,1,0,93212.89,Type_ADept_92Store_262010-08-27IsHoliday_False,2010-08-20,93212.89,0.0,0.00,93212.89
11,136.183129,2011-08-19,Dept_92,3.820,IsHoliday_False,0.0,0.0,0.0,0.0,0.0,...,0,1,0,93212.89,Type_ADept_92Store_262011-08-19IsHoliday_False,2011-08-12,93212.89,0.0,0.00,93212.89


In [26]:
np.random.seed(42)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train[selector], train['Difference'], test_size=0.2, random_state=42)


In [34]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=20)
model.fit(X_train, y_train)
#print(y_train)

ValueError: Unknown label type: 'continuous'